<a href="https://colab.research.google.com/github/futureCodersSE/data-roles/blob/main/Data_Roles_job_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libaries and defining base jobsite urls

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'  ##<- this is really annoying, I WANT to copy the DF.
import numpy as np
import datetime
import matplotlib.pyplot as plt
from google.colab import files

TODO: break down the URL's to allow for user imput


user imput to download/save.

pl=osgb4000000074565668 -> Ashford   
pl=osgb4000000074565139 -> Canterbury   
pl=osgb4000000074543685 -> Tenterden   

In [5]:
#"https://www.reed.co.uk/jobs/"+search1+"-"+search2+"jobs-in"+area+"?proximity="+radius -----?keywords=search1-search2
#"https://jobsinkent.com/search?q=" + search + "&pl=1&r=" + radius
#%2C for a ',' and + for a " "


reed_url = "https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?proximity=30"
jobsinkent_url = "https://jobsinkent.com/search?q=Data&pl=1" #pl=1 -> Kent

max_job_call = 2000 #reed has a limit of 2000 job search api calls per hour.
date_today = datetime.date.today()

General Functions that are used in both searches.

In [6]:
#returns the full HTML of 
def get_html(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup
#This cleans the dataframes when they have been created.
def general_clean(df,site):
  df.drop_duplicates(inplace=True)
  df = df[df["job_title"].str.contains("Data")]
  df["salary"].replace(np.nan, '', regex=False,inplace=True) #not all companies have listed a salary and so NaN is sometimes returned
  df["company"] = df["company"].str.lstrip(" ").str.rstrip(" ") #some companies have excess whitespaces around their names

  df["date_found"] = datetime.date.today()
  df["advertised_on"] = site
  return df

#Reed Job Search
---

This function finds the job ID for all jobs displayed on each page url from reed.co.uk


In [7]:
def find__reed_jobs_iterate(page_url):
  site_html = get_html(page_url)
  results = site_html.find(class_="col-sm-8 col-md-9 results-container")

  id_list = np.array([])
  job_cards = results.find_all(class_="job-result-card")

  for job in job_cards: #finds the jobs, gets it's ID, returns the id
      job_id = int(job["id"].split("jobSection")[1]) #jobSection48529572 ect...
      
      id_list = np.append(id_list,[job_id])
  return id_list

This finds the total jobs found in the search

In [8]:
def find_total_jobs(site_html,max_job_call):
  max_page_requests = int(max_job_call/25) 

  ###
  total_jobs_text = site_html.find(class_="col-sm-11 col-xs-12 page-title").text # '\n' '\r' '\n' x,xxx\r\n  Data Jobs near Ashford       '\n'....
  total_jobs_text = total_jobs_text.replace("\n","").replace("\r","").replace(",","") #          xxxx         Data Jobs near Ashford         

  total_jobs = int(total_jobs_text.split("Data")[0].strip(" "))
  total_pages = int(np.ceil(total_jobs/25))

  #limits the requests to only 2000
  if total_pages > max_page_requests:
    total_pages = max_page_requests


  return total_jobs, total_pages
#find_total_jobs(get_html(reed_url),max_job_call)



The function srapes website for all job id's that meet the search critiera.


In [9]:
def reed_scrape(base_url,max_job_call):
  soup = get_html(base_url)
  total_id_list = np.array([],dtype=int)

  #finds out how many more pages to scrape
  total_jobs, total_pages = find_total_jobs(soup,max_job_call)

  print("total jobs found: ", total_jobs)
  print("total pages: ", total_pages)
  print("pages to search: ",total_pages)

  for page_no in np.arange(1,total_pages+1): 
    page_url = base_url+"&pageno="+str(page_no)
    page_id_list = find__reed_jobs_iterate(page_url)
    total_id_list = np.append(total_id_list,[page_id_list])

    #user feedback
    print("page ",page_no, " processed")
  return total_id_list


This calls the reed API to return job information based on the jobID

In [10]:
def call_reed_api(id_list):
  base_url = "https://www.reed.co.uk/api/1.0/jobs/"
  api_url = ""
  job_df = pd.DataFrame()


  for job_id in id_list:
    api_url = base_url+str(job_id)
    post_request = requests.get(api_url, auth=("96a7ec49-549c-4529-b9d2-fa3059a437b3",""))
    json_data = post_request.json()

    new_row = pd.json_normalize(json_data)
    job_df = job_df.append(new_row,ignore_index=True)
  return job_df

Calls the scraping function and the api function in turn -7mins to run

In [11]:
reed_id_list = reed_scrape(reed_url,max_job_call)
reed_id_list = reed_id_list.astype("int") #it really didnt want to save the array as an int

full_reed_df = call_reed_api(reed_id_list)
display(full_reed_df.info())

total jobs found:  1502
total pages:  61
pages to search:  61
page  1  processed
page  2  processed
page  3  processed
page  4  processed
page  5  processed
page  6  processed
page  7  processed
page  8  processed
page  9  processed
page  10  processed
page  11  processed
page  12  processed
page  13  processed
page  14  processed
page  15  processed
page  16  processed
page  17  processed
page  18  processed
page  19  processed
page  20  processed
page  21  processed
page  22  processed
page  23  processed
page  24  processed
page  25  processed
page  26  processed
page  27  processed
page  28  processed
page  29  processed
page  30  processed
page  31  processed
page  32  processed
page  33  processed
page  34  processed
page  35  processed
page  36  processed
page  37  processed
page  38  processed
page  39  processed
page  40  processed
page  41  processed
page  42  processed
page  43  processed
page  44  processed
page  45  processed
page  46  processed
page  47  processed
page  4

None

Cleaning the Reed DF

In [12]:
def clean_reed(df_in): #this filters out the necessary rows and renames them so the cleaning function works.
  df = df_in[["jobTitle", "employerName", "salary", "contractType", "locationName",  "jobUrl"]]

  df.rename(columns={"jobTitle":"job_title", "employerName": "company", "contractType":"contract", "locationName":"location",  "jobUrl":"job_url"}, inplace=True)
  return df


reed_df = clean_reed(full_reed_df)
reed_df = general_clean(reed_df,"https://www.reed.co.uk")
display(reed_df.head(5))

,job_title,company,salary,contract,location,job_url,date_found,advertised_on
0,Data Architect,Veolia UK,"£65,000 - £84,600 per annum, inc benefits",Permanent,United Kingdom,https://www.reed.co.uk/jobs/data-architect/486...,2022-10-31,https://www.reed.co.uk
2,Data Analyst,Premier IT,"£30,000 - £35,000 per annum",Permanent,Basildon,https://www.reed.co.uk/jobs/data-analyst/48631725,2022-10-31,https://www.reed.co.uk
3,Data Analyst,Morgan Sindall Property Services,"£26,000 - £34,000 per annum, inc benefits",Permanent,Billericay,https://www.reed.co.uk/jobs/data-analyst/48223150,2022-10-31,https://www.reed.co.uk
4,Data Analyst,IronmongeryDirect & ElectricalDirect,"£32,000 - £35,000 per annum",Permanent,Basildon,https://www.reed.co.uk/jobs/data-analyst/48320718,2022-10-31,https://www.reed.co.uk
5,Data Analyst,OnetoOne Personnel,"£33,000 - £50,000 per annum",Permanent,Leigh-on-Sea,https://www.reed.co.uk/jobs/data-analyst/48465441,2022-10-31,https://www.reed.co.uk


#JobsInKent Job Search
---


This function returns a DF containing all of the relevent jobs found on 1 page

In [13]:
def find_jobsinkent_iterate(url,job_limit):
  jobs_df = pd.DataFrame(data=[], columns = ["job_title", "company", "salary", "contract", "location","job_url"])
  site_html = get_html(url)
  results = site_html.find(class_="mt-2 lg:mt-0")

  #this is finding an additional 20 jobs per page that are not visible on the site itself.
  job_cards = results.find_all(class_="flex mb-2 border-black-900 rounded-md border-x-4 sm:rounded-lg")


  #[:job_limit] is required as non relevent jobs were being found in the site_html with the job_cards class after the 20~ jobs that were being show on the site.
  # So I had to find how many jobs were being show in each page first and then limit the loop to only iterate up to that number of jobs.
  for job in job_cards[:job_limit]:

    job_title_url = job.find("a", class_="text-blue-700 visited:text-grey-200 hover:underline") 
    job_title = job_title_url.text
    job_url = job_title_url["href"]

    company_name = job.find(class_="mt-0 mb-2 max-w-2xl text-sm text-black").text 

    #This info was all on the fouth box class and needed to be seperated further for the relevent information to be extracted.
    info_list = job.find_all(class_="col-span-12") 
    salary = clean_bloat(info_list[0].text)
    contract = clean_bloat(info_list[1].text)
    location = clean_bloat(info_list[2].text)

    new_row = {"job_title":job_title, "company":company_name, "salary":salary, "contract":contract, "location":location,"job_url":job_url}
    jobs_df = jobs_df.append(new_row,ignore_index = True)
  return jobs_df

In [14]:
#removes '\n' and spaces from from names.
def clean_bloat(text):
  strip1 = text.replace("\n","").replace("\xa0","")
  strip2 = strip1.strip(" ")
  return strip2

#extracts information about numbers of jobs 
def return_jobs_pages(input_str):
  p_j = input_str.split("of") #[showing 1 to yy] , [xx jobs]
  jobs_per_page = int(p_j[0].split("to")[1]) #[yy]
  total_jobs = int(p_j[1].split("Jobs")[0]) #[xx]
  
  total_pages = int(np.ceil(total_jobs/jobs_per_page))

  return total_jobs, jobs_per_page, total_pages

The main function that calls to find out how many pages are needed to search over and stores the results of each iteration call.

In [15]:
def jobsinkent_scrape(base_url):
  #initialse the DF that will be used
  jobsinkent_df = pd.DataFrame(data=[], columns = ["job_title", "company", "salary", "contract", "location","job_url"])

  #returns ALL the html from the site
  site_html = get_html(base_url)

  banner = site_html.find("p", class_="text-sm text-gray-700 leading-5") #~6th child within the top banner.
  showing = banner.text # 'Showing 1 to yy of xx Jobs'

  total_jobs, jobs_per_page, total_pages = return_jobs_pages(showing)

  for page_no in range(1,total_pages+1):
    #this if statment is required due to the issues mentioned above about job limits. It sets the jobs_limit variable to however many jobs there are displayed (visibly) on the page.
    if page_no == total_pages:
      jobs_limit = total_jobs%jobs_per_page
    else:
      jobs_limit = jobs_per_page

    page_url = base_url + "&page=" + str(page_no)
    new_df = find_jobsinkent_iterate(page_url, jobs_limit)
    jobsinkent_df = jobsinkent_df.append(new_df, ignore_index=True)
  
    #user feedback  
    print("page ",page_no, " processed")

  return jobsinkent_df
  
jobsinkent_df = jobsinkent_scrape(jobsinkent_url)
display(jobsinkent_df.head(5))

page  1  processed
page  2  processed
page  3  processed


,job_title,company,salary,contract,location,job_url
0,Senior Data Analyst,Clarion Housing Group Ltd,£43264 - £59488 per annum,"permanent,full-time","Central London, Greater London",https://jobsinkent.com/job/2423530
1,Data Analyst - Global Insurance Markets,MW Appointments,Negotiable,"permanent,full-time","City of London, Greater London",https://jobsinkent.com/job/2422053
2,Data Analyst,Pearson Whiffin Recruitment,£23000 - £28000 per annum + DOE,"permanent,full-time","West Malling, Kent",https://jobsinkent.com/job/2331144
3,Data Lead (Digital Marketing),Recruitment Solutions Folkestone Ltd,"£30-40,000 dep on exp","permanent,full-time","Ashford, Kent",https://jobsinkent.com/job/2277249
4,Data & GDPR Officer,Pearson Whiffin Recruitment,"Up to £35,000 per annum","permanent,full-time","Kings Hill, Kent",https://jobsinkent.com/job/2424610


Cleans the jobsinkent_df to be used later.

In [16]:
jobsinkent_df = general_clean(jobsinkent_df,"jobsinkent.com/search")

#Combining both results
---


Combines the two DF's from JobsInKent

In [17]:
data_jobs_df = pd.DataFrame()

data_jobs_df = data_jobs_df.append([jobsinkent_df,reed_df],ignore_index=True)
display(data_jobs_df)

,job_title,company,salary,contract,location,job_url,date_found,advertised_on
0,Senior Data Analyst,Clarion Housing Group Ltd,£43264 - £59488 per annum,"permanent,full-time","Central London, Greater London",https://jobsinkent.com/job/2423530,2022-10-31,jobsinkent.com/search
1,Data Analyst - Global Insurance Markets,MW Appointments,Negotiable,"permanent,full-time","City of London, Greater London",https://jobsinkent.com/job/2422053,2022-10-31,jobsinkent.com/search
2,Data Analyst,Pearson Whiffin Recruitment,£23000 - £28000 per annum + DOE,"permanent,full-time","West Malling, Kent",https://jobsinkent.com/job/2331144,2022-10-31,jobsinkent.com/search
3,Data Lead (Digital Marketing),Recruitment Solutions Folkestone Ltd,"£30-40,000 dep on exp","permanent,full-time","Ashford, Kent",https://jobsinkent.com/job/2277249,2022-10-31,jobsinkent.com/search
4,Data & GDPR Officer,Pearson Whiffin Recruitment,"Up to £35,000 per annum","permanent,full-time","Kings Hill, Kent",https://jobsinkent.com/job/2424610,2022-10-31,jobsinkent.com/search
...,...,...,...,...,...,...,...,...
103,Exams & Data Officer - Bromley / Kent Borders ...,Randstad Education,"£23,718 - £25,170 per annum",Permanent,Swanley,https://www.reed.co.uk/jobs/exams-data-officer...,2022-10-31,https://www.reed.co.uk
104,Digital Product Owner (Data Products),SS&C,,Permanent,Basildon,https://www.reed.co.uk/jobs/digital-product-ow...,2022-10-31,https://www.reed.co.uk
105,Digital Analyst/Data Visualisation Executive- ...,Recruitment Solutions,"£30,000 - £35,000 per annum",Permanent,Tunbridge Wells,https://www.reed.co.uk/jobs/digital-analyst-da...,2022-10-31,https://www.reed.co.uk
106,Temporary Database Administrator,Julie Rose Recruitment,£11.00 - £13.00 per hour,Temporary,Sevenoaks,https://www.reed.co.uk/jobs/temporary-database...,2022-10-31,https://www.reed.co.uk


This returns a **unique** list of recruiters and compaies who are advertiseing data roles

In [18]:
def find_company_lists():
  data_jobs_employer_names = data_jobs_df.drop_duplicates(subset = "company") #data_jobs_df["company"].unique()

  #seperating out all recruitment companies from the list
  companies_df = data_jobs_employer_names.drop( data_jobs_employer_names[ data_jobs_employer_names["company"].str.contains("Recruit")==True].index)

  #This requires going through each company and deciding if they are a recruitment agency or not if they dont have 'Recruit' in their name and adding them to the list.
  additional_recruiter_list = ["Reed","OnetoOne Personnel","Huntress","MW Appointments","P3 Search & Selection","Manpower - Ashford","Morgan McKinley","Brook Street","McGregor Boyall","SAGA","Office Angels","REED",
                            "Commercial Services Interim & Executive Search","Harnham","Connect2Staff","Senitor Associates","Zorba Consulting Limited","GerrardWhite","Lorien","Morgan Law", "Academics","eFinancialCareers",
                            "Commercial Services HR", "OBJECTIVE HR LTD", ]
  it_specialits = ["Senitor Associates","Zorba Consulting Limited","GerrardWhite","Lorien","eFinancialCareers"]

  companies_df = companies_df.drop( companies_df[companies_df["company"].isin(additional_recruiter_list) == True ].index)
  companies_list = companies_df["company"]

  recruiters_df1 = data_jobs_employer_names.drop( data_jobs_employer_names[ data_jobs_employer_names["company"].str.contains("Recruit")==False].index) #drop 'Recruit' is not in name
  recruiters_df2 = data_jobs_employer_names.drop( data_jobs_employer_names[data_jobs_employer_names["company"].isin(additional_recruiter_list) == False ].index) #drop if company is not in 'additional_drop_list'

  recruiters_df = recruiters_df1.append(recruiters_df2,ignore_index=True) #combine the two sets of lists
  recruiters_list = recruiters_df["company"]

  return companies_list, recruiters_list

companies_list, recruiters_list = find_company_lists()

In [22]:
display(companies_list)

0                 Clarion Housing Group Ltd
5                           PW Construction
10          Commercial Services Trading Ltd
54                                Veolia UK
55                               Premier IT
56         Morgan Sindall Property Services
57     IronmongeryDirect & ElectricalDirect
62             Braundton Consulting Limited
65              TELUS International AI Inc.
66                              P&O Ferries
67                Nigel Frank International
70                                    Hatch
72                           Shorterm Group
74                       F&D Tech Solutions
75                          Hastings Direct
79                        Harris Federation
80                                  Pontoon
82                        Howells Solutions
88                        Rutherford Briant
94                              925HomeJobs
103                      Randstad Education
104                                    SS&C
Name: company, dtype: object

This takes the lists of unique compaies and recruiters and returns a DataFrame with **all** the roles they have advertised.

In [23]:
def find_company_df():
  company_df = data_jobs_df.drop( data_jobs_df[data_jobs_df["company"].isin(recruiters_list) == True ].index) #drop if company is a recruiter
  recruiters_df = data_jobs_df.drop( data_jobs_df[data_jobs_df["company"].isin(companies_list) == True ].index) #drop if company is not a recruiter

  #For readability.
  company_df.reset_index(drop=True, inplace = True)
  recruiters_df.reset_index(drop=True, inplace = True)

  return company_df, recruiters_df

company_df, recruiters_df = find_company_df()
  

To download and save the DataFrames, uncomment save() and download() and run the cell

In [24]:
#saveing files names of search based on current date.
recruiters_fn = str(date_today)+"_recruitment_company_df.csv"
company_fn = str(date_today)+"_data_employeer_df.csv"

#creats a .csv file in colab (cloud?) NB: This does not save the .csv files to you PC, you need to download them first.
def save():
  recruiters_df.to_csv(recruiters_fn)
  company_df.to_csv(company_fn)

#downloads files to pc
def download():
  files.download(recruiters_fn)
  files.download(company_fn)

###Uncomment to save and download, commented by default.

In [25]:
#save()
#download()